In [ ]:
import math
import ipywidgets as widgets
import pandas_datareader as web
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cufflinks as cf
from IPython.display import display
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import MinMaxScaler

**INDONESIA's STOCK EXCHANGE MOVING AVERAGE PREDICTION**

---



In [ ]:
## PARAMETERS
tickerx=widgets.Text(
    value='BBCA.JK',
    placeholder='TICKER IDX',
    description='KODE TICKER SAHAM:',
    disabled=False   
)
ticker2='BBCA.JK'

samplingx=widgets.IntSlider(
    value=60,
    min=0,
    max=100,
    step=1,
    description='SAMPLING DATA:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

training_lenx=widgets.FloatSlider(
    value=0.4,
    min=0,
    max=1.0,
    step=0.1,
    description='% TRAINING DATA:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

start_date='1980-01-01'
end_date=datetime.today().strftime('%Y-%m-%d')
#end_date='2020-05-26'

moving_averagex=widgets.IntSlider(
    value=5,
    min=0,
    max=120,
    step=1,
    description='MOVING AVERAGE:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

In [ ]:
def menuawal (ticker,moving_average):
  df=web.DataReader(ticker,data_source='yahoo', end=end_date)
  #qf=cf.QuantFig(df,title=ticker,legend='top',name='GS')
  #qf.add_sma([moving_average,20],width=2,color=['RED','BLACK'],legendgroup=True)
  #qf.add_volume()
  #qf.iplot(asFigure=True).show()
  df['Close'].iplot(asFigure=True, title=ticker + ' CLOSE PRICE',yTitle="HARGA PENUTUPAN (Rp)", xTitle="TANGGAL").show()
  
widgets.interact(menuawal,ticker=tickerx,moving_average=moving_averagex)


interactive(children=(Text(value='BBCA.JK', description='KODE TICKER SAHAM:', placeholder='TICKER IDX'), IntSl…

<function __main__.menuawal>

In [ ]:
 def belajar (ticker,moving_average,sampling, training_len): 
  df=web.DataReader(ticker,data_source='yahoo', end=end_date)
  df['SMA1']=df['Close'].rolling(window=moving_average).mean()
  df=df[moving_average-1:]

  # convert the data inti numpy array
  data=df.filter(['SMA1'])
  dataset=data.values

  # get the number of rows to train the model on
  training_data_len=math.ceil(len(dataset)* training_len)

  scaler = MinMaxScaler(feature_range=(0,1))
  scaled_data = scaler.fit_transform(dataset)

  # SPLIT DATA INTO x_train and y_train
  train_data=scaled_data[0:training_data_len]
  x_train=[]
  y_train=[]

  for i in range(sampling,len(train_data)):
    x_train.append(train_data[i-sampling:i,0])
    y_train.append(train_data[i,0])

  x_train, y_train=np.array(x_train), np.array(y_train)

  # SPLIT DATA INTO x_test and y_test
  test_data=scaled_data[training_data_len-sampling:]

  x_test=[]
  y_test=dataset[training_data_len+1:]

  for i in range(sampling, len(test_data)):
    x_test.append(test_data[i-sampling:i,0]) 

  lr=LinearRegression()
  lr.fit(x_train,y_train)

  predictions= lr.predict(x_test)
  predictions=np.reshape(predictions,(predictions.shape[0],1))
  predictions=scaler.inverse_transform(predictions)

  df_baru=df[training_data_len:]
  df_baru['prediction']=predictions
  df_baru[['SMA1','prediction']].iplot(asFigure=True,title=ticker,legend='top').show()

  close=df_baru.filter(['SMA1']).values
  predict=df_baru.filter(['prediction']).values
  rmse =np.sqrt(np.mean(((close-predict)**2)))
  print('PERSEN ERROR PREDIKSI SMA = {:.2f}%'.format(rmse))

  new_df=df.filter(['SMA1'])
  last_60_days=new_df[-sampling:].values
  last_data=scaler.transform(last_60_days)
  last_data=np.reshape(last_data,(1,len(last_data)))
  last_prediction=lr.predict(last_data)
  last_prediction=[last_prediction]
  last_prediction=scaler.inverse_transform(last_prediction)

  print('PREDIKSI SMA ({}) BESOK = {}'.format(end_date,last_prediction[0][0]))


widgets.interact(belajar,ticker=tickerx,moving_average=moving_averagex,sampling=samplingx, training_len=training_lenx)


interactive(children=(Text(value='BBCA.JK', description='KODE TICKER SAHAM:', placeholder='TICKER IDX'), IntSl…

<function __main__.belajar>